In [1]:
import numpy as np
DTYPE = np.float64
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
import sys
sys.path.insert(0, "../../packages")
import flory_
import math

In [2]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [3]:
# General Function for computing mergers of compartments
# Returns the concentrations of components in the merged and the unmerged compartments
# Stores them in phi_in_kmerged and phi_in_kunmerged respectively.


def mergers(concs:np.array, vols:np.array, chis, merged_compartments:list):
    # Find the unmerged compartment(s)
    expected = len(vols)*(len(vols)+1)//2
    actual = np.sum(merged_compartments)
    unmerged_compartment = expected - actual

    # print(concs)
    # print(vols)
    # print(unmerged)

    # Compute the merged volumes, stored in variable eta_merged
    # subtract -1 from the merged_compartments idxs to maintain python idxing
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]
    # print(eta_merged)

    # Calculating the compositions of components in the merged compartment
    # Taking a simple weighted average
    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1])/eta_merged
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1])/eta_merged
    phi_3merged = 1 - phi_1merged - phi_2merged
    # print(phi_1merged, phi_2merged, phi_3merged)
    # print(phi_1merged + phi_2merged + phi_3merged)

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged]
    phi_in_kunmerged = [concs[0, unmerged_compartment-1], concs[1, unmerged_compartment-1], concs[2, unmerged_compartment-1]]
    # print(phi_in_kmerged)
    # print(phi_in_kunmerged)

    F_merged = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged_compartment-1]*floryHuggins(phi_in_kunmerged, chis)
    # print(F_merged)

    return phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment

    

In [4]:
# Global concentrations
phi_global = np.array([0.3, 0.3, 0.4], dtype = DTYPE)

# Xs = np.arange(0, 10.1, 0.1)
# Xs = [1]
# tol = DTYPE(1e-4)
x = 3.6

In [5]:
chis = np.array([[0, 3.0, 3+x], [3.0, 0.0, 3.0], [3+x, 3.0, 0.0]], dtype = DTYPE)

In [6]:
n_components = 3
free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
interaction = free_energy.interaction
entropy = free_energy.entropy
ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
options = {
    "num_part": 32,
    "progress": False,
    "max_steps": 1000000,  # disable progress bar, allow more steps
}

finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
phases = finder.run(progress=False)


In [7]:
vols = phases.volumes
concs = phases.fractions
vols, concs


(array([1.53027643, 0.74411444, 1.18889242, 1.12783048, 1.21053801,
        0.77532392, 0.75288253, 1.03704916, 0.60585441, 1.0279823 ,
        0.95714044, 1.24054868, 1.53511862, 1.16068925, 1.29755613,
        1.09413325, 0.78554185, 0.75227991, 1.15516466, 0.75796518,
        1.4351309 , 1.09217739, 0.71293882, 1.55434863, 0.64771553,
        0.91489131, 0.74731572, 0.82723581, 1.02240084, 0.44232069,
        0.85775684, 1.00888544]),
 array([[0.00200812, 0.06559645, 0.93239539],
        [0.9323954 , 0.06559644, 0.00200812],
        [0.9323954 , 0.06559644, 0.00200812],
        [0.00200812, 0.06559645, 0.93239539],
        [0.06550498, 0.86899016, 0.06550496],
        [0.06550498, 0.86899016, 0.06550496],
        [0.00200812, 0.06559645, 0.93239539],
        [0.9323954 , 0.06559644, 0.00200812],
        [0.06550498, 0.86899016, 0.06550496],
        [0.9323954 , 0.06559644, 0.00200812],
        [0.00200812, 0.06559645, 0.93239539],
        [0.00200812, 0.06559645, 0.93239539],
      

In [8]:
vols = phases.get_clusters().volumes
concs = phases.get_clusters().fractions
n_components = 3
free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
interaction = free_energy.interaction
entropy = free_energy.entropy
ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
options = {
    "num_part": 32,
    "progress": False,
    "max_steps": 1000000,  # disable progress bar, allow more steps
}

finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)

vols = vols/np.sum(vols)
vols, concs

(array([0.30037557, 0.4078577 , 0.29176673]),
 array([[0.9323954 , 0.06559644, 0.00200812],
        [0.00200812, 0.06559645, 0.93239539],
        [0.06550498, 0.86899016, 0.06550496]]))

In [29]:
# 334
Xs=  [0.3]

In [30]:
phi_global = np.array([0.3, 0.3, 0.4], dtype = DTYPE)
n_components = 3

for X in (Xs):
    chis = np.array([[0, 3.0, 3+X], [3.0, 0.0, 3.0], [3+X, 3.0, 0.0]], dtype = DTYPE)

    free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
    interaction = free_energy.interaction
    entropy = free_energy.entropy
    ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
    options = {
    "num_part": 32,
    "progress": False,
    "max_steps": 1000000,  # disable progress bar, allow more steps
    }
    
    finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
    phases = finder.run(progress=False)
    
    vols = phases.get_clusters().volumes
    vols = vols/np.sum(vols) # normalization
    
    concs = phases.get_clusters().fractions

    print("------------------------------------------------------------------------")
    print("------------------------------------------------------------------------")
    print(f"X = {X}")
    print(concs)
    print(vols)
    print("------------------------------------------------------------------------")
    
    merged_compartments_list = [np.array([1, 3], dtype=np.int64), np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64)]
    for merged_compartments in merged_compartments_list:
        phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers(concs, vols, chis, merged_compartments)
        # print(unmerged_compartment)
        phi_global_ = np.array([phi_in_kunmerged, phi_in_kmerged])
        # print(phi_global_)
        # print()
        vols_ = np.array([1-eta_merged, eta_merged])
        options = {
        "num_part": 2,
        "progress": False,
        "max_steps": 100000,  # disable progress bar, allow more steps
        }
        finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        finder2.reinitialize_from_omegas(-np.log(np.transpose(phi_global_)), vols_)
        # finder2.reinitialize_from_phis(np.transpose(phi_global_), vols_)
        phases = finder2.run(progress=False).get_clusters()
    
        print(f"merged compartments {merged_compartments}")
        print(f"vols {phases.volumes/np.sum(phases.volumes)}")
        print(f"fracs {phases.fractions}")
        print()

------------------------------------------------------------------------
------------------------------------------------------------------------
X = 0.3
[[0.06306834 0.08498295 0.85194867]
 [0.85194779 0.08498339 0.0630688 ]
 [0.09334706 0.81330571 0.09334732]]
[0.41576966 0.28900831 0.29522203]
------------------------------------------------------------------------
merged compartments [1 3]
vols [0.3522676 0.6477324]
fracs [[0.0753011  0.09931876 0.82538263]
 [0.42220193 0.40913999 0.16865673]]

merged compartments [1 2]
vols [0.23531846 0.76468154]
fracs [[0.80289486 0.1184815  0.07875273]
 [0.14524154 0.35585963 0.4988591 ]]

merged compartments [2 3]
vols [0.63509219 0.36490781]
fracs [[0.42693166 0.41201965 0.16101002]
 [0.07908639 0.10503946 0.81594145]]



In [19]:
Xs = [8]

In [20]:
phi_global = np.array([0.2, 0.3, 0.5], dtype = DTYPE)

for X in (Xs):
    chis = np.array([[0, 3.0, 3+X], [3.0, 0.0, 3.0], [3+X, 3.0, 0.0]], dtype = DTYPE)
    phases = finder.run(progress=False)
    vols = phases.volumes
    concs = phases.fractions
    vols = phases.get_clusters().volumes
    concs = phases.get_clusters().fractions
    n_components = 3
    free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
    interaction = free_energy.interaction
    entropy = free_energy.entropy
    ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
    options = {
        "num_part": 32,
        "progress": False,
        "max_steps": 1000000,  # disable progress bar, allow more steps
    }
    
    finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
    
    vols = vols/np.sum(vols)


    print("------------------------------------------------------------------------")
    print("------------------------------------------------------------------------")
    print(f"X = {X}")
    print(concs)
    print(vols)
    print("------------------------------------------------------------------------")
    
    merged_compartments_list = [np.array([1, 3], dtype=np.int64), np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64)]
    for merged_compartments in merged_compartments_list:
        phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers(concs, vols, chis, merged_compartments)
        # print(unmerged_compartment)
        phi_global_ = np.array([phi_in_kunmerged, phi_in_kmerged])
        print(f"guess {phi_global_}")
        
        vols_ = np.array([1-eta_merged, eta_merged])
        options = {
        "num_part": 2,
        "progress": False,
        "max_steps": 1000000,  # disable progress bar, allow more steps
        }
        finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
        finder2.reinitialize_from_phis(np.transpose(phi_global_), vols_)
        phases = finder2.run(progress=False).get_clusters()
    
        print(merged_compartments)
        print(phases.volumes/np.sum(phases.volumes))
        print(phases.fractions)
        print()


------------------------------------------------------------------------
------------------------------------------------------------------------
X = 8
[[0.09453826 0.09453969 0.81092203]
 [0.81091016 0.09454467 0.09454527]
 [0.09453974 0.8109185  0.09454177]]
[0.56598114 0.14721587 0.28680299]
------------------------------------------------------------------------
guess [[0.09453969 0.09454467 0.8109185 ]
 [0.33546794 0.56998683 0.09454523]]
[1 3]
[0.53961208 0.46038792]
[[4.33815291e-05 7.60155146e-02 9.23941055e-01]
 [4.34365414e-01 5.62528027e-01 3.10661652e-03]]

guess [[0.81092203 0.09454527 0.09454177]
 [0.09453855 0.66304026 0.24242119]]
[1 2]
[0.46038713 0.53961287]
[[4.34366158e-01 5.62527194e-01 3.10656711e-03]
 [4.33825370e-05 7.60169380e-02 9.23939748e-01]]

guess [[0.09453826 0.81091016 0.09453974]
 [0.56793062 0.09454507 0.33752431]]
[2 3]
[0.53961417 0.46038583]
[[4.33842002e-05 7.60192867e-02 9.23937593e-01]
 [4.34367386e-01 5.62525819e-01 3.10648558e-03]]

